In [75]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [76]:
!pip install catboost sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.5 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 19.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [77]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from catboost import CatBoostClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

In [79]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [80]:
# 결측치 처리
text_columns = ["customer_type", "customer_job", "inquiry_type", "product_category",
                "product_subcategory", "product_modelname", "customer_position",
                "response_corporate", "expected_timeline", "business_area"]

df_train[text_columns] = df_train[text_columns].fillna('')
df_test[text_columns] = df_test[text_columns].fillna('')

# 자연어 데이터 통합
df_train['text_data'] = df_train[text_columns].astype(str).apply(lambda x: ' '.join(x), axis=1)
df_test['text_data'] = df_test[text_columns].astype(str).apply(lambda x: ' '.join(x), axis=1)

In [81]:
# 미리 훈련된 sentence transformer 모델 로드
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

config_sentence_transformers.json: 100%|██████████| 122/122 [00:00<00:00, 251kB/s]
README.md: 100%|██████████| 3.73k/3.73k [00:00<00:00, 8.15MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 108kB/s]
config.json: 100%|██████████| 629/629 [00:00<00:00, 1.39MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:00<00:00, 131MB/s] 
tokenizer_config.json: 100%|██████████| 314/314 [00:00<00:00, 1.11MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.24MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.23MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 704kB/s]


In [82]:
# 텍스트 데이터에 대한 토큰화 및 임베딩 생성
def create_embeddings(sentences):
    return model.encode(sentences, convert_to_tensor=True)

# 결측치를 유사도를 사용하여 채우는 함수
def fill_missing_with_similarity(df, text_column):
    # 결측치와 결측치가 아닌 값으로 행 분리
    missing_rows = df[df[text_column].isnull()]
    non_missing_rows = df.dropna(subset=[text_column])

    # 결측치가 아닌 값에 대한 임베딩 생성
    embeddings = create_embeddings(non_missing_rows[text_column].tolist())

    # 결측치가 있는 행을 반복하고 가장 유사한 결측치가 없는 값을 사용하여 채우기
    for index, missing_row in missing_rows.iterrows():
        missing_embedding = create_embeddings([missing_row[text_column]])[0]
        similarities = cosine_similarity([missing_embedding], embeddings)[0]
        most_similar_index = similarities.argmax()

        # 결측치를 가장 유사한 결측치가 없는 값으로 채우기
        df.at[index, text_column] = non_missing_rows.iloc[most_similar_index][text_column]

# 'text_data' 열에서 결측치를 채우기 위해 함수 적용
fill_missing_with_similarity(df_train, 'text_data')
fill_missing_with_similarity(df_test, 'text_data')

In [83]:
# TF-IDF 변환
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_train = tfidf_vectorizer.fit_transform(df_train['text_data'])
tfidf_test = tfidf_vectorizer.transform(df_test['text_data'])

In [84]:
# TruncatedSVD를 사용하여 차원 축소
svd = TruncatedSVD(n_components=50, random_state=42)
svd_train = svd.fit_transform(tfidf_train)
svd_test = svd.transform(tfidf_test)

In [85]:
# 변환된 데이터를 데이터프레임에 추가
df_svd_train = pd.DataFrame(svd_train, columns=[f'svd_{i}' for i in range(svd_train.shape[1])])
df_svd_test = pd.DataFrame(svd_test, columns=[f'svd_{i}' for i in range(svd_test.shape[1])])

In [86]:
df_train = pd.concat([df_train, df_svd_train], axis=1)
df_test = pd.concat([df_test, df_svd_test], axis=1)

In [87]:
# 원래의 텍스트 열 및 중간 열들 삭제
df_train.drop(text_columns + ['text_data'], axis=1, inplace=True)
df_test.drop(text_columns + ['text_data'], axis=1, inplace=True)

In [88]:
# 수치형 변수 전체 Standardization, Normalization 실행
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()
columns_to_ST = ["com_reg_ver_win_rate", "historical_existing_cnt", "lead_desc_length"]
columns_to_NM = ["ver_win_rate_x", "ver_win_ratio_per_bu"]

In [89]:
# Apply Standardization to each column
for column in columns_to_ST:
    df_train[column + '_standardized'] = scaler_standard.fit_transform(df_train[[column]])
    df_test[column + '_standardized'] = scaler_standard.transform(df_test[[column]])
    df_train.drop(columns=[column], inplace=True)
    df_test.drop(columns=[column], inplace=True)

# Apply Normalization to each column
for column in columns_to_NM:
    df_train[column + '_normalized'] = scaler_minmax.fit_transform(df_train[[column]])
    df_test[column + '_normalized'] = scaler_minmax.transform(df_test[[column]])
    df_train.drop(columns=[column], inplace=True)
    df_test.drop(columns=[column], inplace=True)

In [90]:
# 훈련 데이터와 검증 데이터로 분할
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

cat_features_col = list(x_train.columns)
cat_features_col.remove("bant_submit")
cat_features_col.remove("com_reg_ver_win_rate_standardized")
cat_features_col.remove("historical_existing_cnt_standardized")
cat_features_col.remove("id_strategic_ver")
cat_features_col.remove("it_strategic_ver")
cat_features_col.remove("idit_strategic_ver")
cat_features_col.remove("lead_desc_length_standardized")
cat_features_col.remove("ver_cus")
cat_features_col.remove("ver_pro")
cat_features_col.remove("ver_win_rate_x_normalized")
cat_features_col.remove("ver_win_ratio_per_bu_normalized")

for i in range(50):
  cat_features_col.remove("svd_"+str(i))

In [91]:
catboost_model = CatBoostClassifier(
    verbose=0,
    iterations=1000,
    depth=6,
    learning_rate=0.1,
    loss_function='Logloss',
    one_hot_max_size=5,
    cat_features=cat_features_col
)

In [92]:
# Calculate the most frequent value (mode) for each column
modes = x_train.mode().iloc[0]

# Fill missing (NA) values with the mode
x_train.fillna(modes, inplace=True)

In [93]:
catboost_model.fit(x_train, y_train)

In [94]:
# Calculate the most frequent value (mode) for each column
modes = x_val.mode().iloc[0]

# Fill missing (NA) values with the mode
x_val.fillna(modes, inplace=True)

In [95]:
# 검증 데이터에 대한 예측 및 성능 평가
y_val_pred = catboost_model.predict(x_val)
y_val_pred = np.array([True if pred == 'True' else False for pred in y_val_pred])

print("Classification Report:\n", classification_report(y_val, y_val_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred, labels=[True, False]))
print("\nAccuracy: {:.4f}".format(accuracy_score(y_val, y_val_pred)))
print("Precision: {:.4f}".format(precision_score(y_val, y_val_pred, labels=[True, False])))
print("Recall: {:.4f}".format(recall_score(y_val, y_val_pred)))
print("F1 Score: {:.4f}".format(f1_score(y_val, y_val_pred)))

Classification Report:
               precision    recall  f1-score   support

       False       0.98      0.99      0.99     10913
        True       0.92      0.79      0.85       947

    accuracy                           0.98     11860
   macro avg       0.95      0.89      0.92     11860
weighted avg       0.98      0.98      0.98     11860

Confusion Matrix:
 [[  750   197]
 [   67 10846]]

Accuracy: 0.9777
Precision: 0.9180
Recall: 0.7920
F1 Score: 0.8503


In [96]:
x_test = df_test.drop(["is_converted", "id"], axis=1)

# Calculate the most frequent value (mode) for each column
modes = x_test.mode().iloc[0]

# Fill missing (NA) values with the mode
x_test.fillna(modes, inplace=True)

test_pred = catboost_model.predict(x_test)

df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)